## Do general imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', None)

## Load Datasets

In [ ]:
from preprocessing.projects import ProjectsPreProcess

issues_df = pd.read_csv('./temp_data/issues.csv')
# types = ['Ticket','Service','Deployment','HD Service','Project']
types = ['Ticket','Deployment','HD Service']

issues_df = issues_df[(issues_df['issue_proj'].str.match('\w{2}\d{2}\w{1,}'))
                      & (issues_df['issue_type'].isin(types))
                      & (issues_df['issue_created'] >= '2022-01-01')
                      & (issues_df['issue_created'] <= '2022-12-31')
                      & pd.notna(issues_df['issue_resolution_date'])]
                      # & (issues_df['issue_priority'] == 'High')]
print(F'Total records after filter %i' % len(issues_df))

ppp = ProjectsPreProcess()
labeled_projects = ppp.pre_process(issues_df=issues_df,
                                       clusters=5,
                                       features=['wf_total_time_mean','issue_comments_count_mean',
                                                              'issue_contr_count_mean', 'processing_steps_mean'])

ppp.merge(issues_df,labeled_projects)
columns = ['processing_steps', 'issue_comments_count', 'issue_contr_count','issue_proj','proj_category']
df = issues_df[columns].copy()

In [ ]:
ax = plt.figure(figsize=(8, 8)).add_subplot(projection='3d')
colors = palette = sns.color_palette("tab10")
categories = df['proj_category'].drop_duplicates().sort_values()

x = ('Countributors Count', 'issue_contr_count',)
y = ('Comments Count', 'issue_comments_count',)
z = ('Processing Steps', 'processing_steps',)

for i, c in enumerate(categories):
    # Plot the 3D surface
    df_c = df[df['proj_category'] == c]
    projects = df_c['issue_proj'].drop_duplicates()
    x_v = df_c[x[1]]
    y_v = df_c[y[1]]
    z_v = df_c[z[1]]
    ax.scatter(x_v, y_v, z_v, c=colors[i], label=f'{len(df_c)} items for {len(projects)} projects')

ax.legend()
if len(z) > 2:
    ax.set_zticks(z[2])
if len(x) > 2:
    ax.set_xticks(x[2])
if len(y) > 2:
    ax.set_yticks(y[2])

# Plot the 3D surface
ax.set_xlabel(x[0])
ax.set_ylabel(y[0])
ax.set_zlabel(z[0])

# ax.view_init(elev=60, azim=50, roll=0)
ax.view_init(elev=30, azim=-30)
ax.set_box_aspect(aspect=None, zoom=0.85)